Метод ближащих соседей (классификация и регрессия), реализованный на Python с использованием библиотек Pandas и NumPy.

In [1]:
import pandas as pd
import numpy as np
import time
import functools

from sklearn.datasets import make_regression, make_classification
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, precision_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

Метод ближайших соседей, регрессия

In [2]:
X, y = make_regression(n_samples=10000, n_features=15, n_informative=10, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
class MyKNNReg:
     """
    Метод ближащих соседей (регрессия)

    Parameters
    ----------
    k : int
        Количество ближайших соседей, by default 3
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: euclidean, chebyshev, manhattan, cosine
    weights : np.ndarray, optional
        Веса модели
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: mae, mse, rmse, mape, r2    
    """
     @staticmethod
     def timer(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            val = func(*args, **kwargs)
            end = time.perf_counter()
            work_time = end - start
            print(f'Время выполнения {func.__name__}: {round(work_time, 4)} сек.')
            return val
        return wrapper   
        
     def __init__(self, k: int = 5, metric: str = 'euclidean', weight: np.array = 'uniform') -> None:
        self.k = k
        self.train_size = None
        self.metric = metric
        self.metrics = {
            'euclidean': lambda test, train: np.sqrt(np.sum((test - train) ** 2, axis=-1)),
            'chebyshev': lambda test, train: np.max(np.abs(test - train), axis=-1),
            'manhattan':  lambda test, train: np.sum(np.abs(test - train), axis=-1),
            'cosine':  lambda test, train: 1 - np.sum(test * train, axis=-1) / (np.sqrt(np.sum(test ** 2, axis=-1)) * (np.sqrt(np.sum(train ** 2, axis=-1))))
         }
        self.weight = weight
        
     def __repr__(self) -> str:
        return f"MyKNNClf class: k={self.k}"
     
     @timer
     def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        """Обучение KNN

        Parameters
        ----------
        X : pd.DataFrame
            Все фичи
        y : pd.Series
            Целевая переменная
        """
        self.X = X.to_numpy()
        self.y = y.to_numpy()
        self.train_size = self.X.shape        
     @timer
     def predict(self, X: pd.DataFrame) -> np.array:
        """Выдача предсказаных значений моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        train = np.expand_dims(self.X, axis=0)
        test = np.expand_dims(X.to_numpy(), axis=1)
        distances = self.metrics[self.metric](train, test)
        indx = np.argsort(distances)[:, :self.k]
        dist = np.sort(distances)[:, :self.k]
        
        return self.__get_proba(indx, distances) 
    
     def __get_proba(self, indx: int, dist: float):
        proba_formulas = {
            "uniform": np.mean(self.y[indx], axis=1),
            "rank": np.sum(self.y[indx] / np.arange(1, self.k + 1), axis=1) / np.sum(1 / np.arange(1, self.k + 1)),
           } 
        
        return proba_formulas[self.weight]


In [4]:
knn = MyKNNReg(k=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(r2_score(y_test, y_pred))

Время выполнения fit: 0.0001 сек.
Время выполнения predict: 2.8526 сек.
0.8028375607764501


In [5]:
y_pred_sklearn = KNeighborsRegressor().fit(X_train, y_train).predict(X_test)
print(r2_score(y_test, y_pred_sklearn))

0.8028375607764501


Метод ближайших соседей, классификация

In [6]:
X, y = make_classification(n_samples=10000, n_features=15, n_informative=10, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
class MyKNNClf:
    """
    Метод ближащих соседей (классификация)

    Parameters
    ----------
    k : int
        Количество ближайших соседей, by default 3
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: euclidean, chebyshev, manhattan, cosine
    weights : np.ndarray, optional
        Веса модели
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: mae, mse, rmse, mape, r2    
    """
    @staticmethod
    def timer(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            val = func(*args, **kwargs)
            end = time.perf_counter()
            work_time = end - start
            print(f'Время выполнения {func.__name__}: {round(work_time, 4)} сек.')
            return val
        return wrapper   
    
    def __init__(self, k: int = 3, metric: str = 'euclidean', weight: np.array = 'uniform'):
        self.k = k
        self.train_size = None
        self.metric = metric
        self.metrics = {
            'euclidean': lambda test, train: np.sqrt(np.sum((test - train) ** 2, axis=-1)),
            'chebyshev': lambda test, train: np.max(np.abs(test - train), axis=-1),
            'manhattan':  lambda test, train: np.sum(np.abs(test - train), axis=-1),
            'cosine':  lambda test, train: 1 - np.sum(test * train, axis=-1) / (np.sqrt(np.sum(test ** 2, axis=-1)) * (np.sqrt(np.sum(train ** 2, axis=-1))))
         }
        self.weight = weight
        
    def __repr__(self) -> str:
        return f"MyKNNClf class: k={self.k}"
    
    @timer
    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        """Обучение KNN

        Parameters
        ----------
        X : pd.DataFrame
            Все фичи
        y : pd.Series
            Целевая переменная
        """
        self.X = X.to_numpy()
        self.y = y.to_numpy()
        self.train_size = self.X.shape   
        
    def predict_proba(self, X: pd.DataFrame) -> np.array:
        """Выдача предсказаных значений моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        train = np.expand_dims(self.X, axis=0)
        test = np.expand_dims(X.to_numpy(), axis=1)
        distances = self.metrics[self.metric](train, test)
        indx = np.argsort(distances)[:, :self.k]
        dist = np.sort(distances)[:, :self.k]
        
        return self.__get_proba(indx, distances) 
    
    @timer
    def predict(self, X: pd.DataFrame) -> np.array:
        """Выдача предсказаных классов моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        return np.array([1 if prob >= 0.5 else 0 for prob in self.predict_proba(X)])
    
    def __get_proba(self, indx: int, dist: float) -> np.array:
        proba_formulas = {
            "uniform": np.mean(self.y[indx], axis=1),
            "rank": np.sum(self.y[indx] / np.arange(1, self.k + 1), axis=1) / np.sum(1 / np.arange(1, self.k + 1))            
        } 
        
        return proba_formulas[self.weight]

In [8]:
knn = MyKNNClf(k=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(precision_score(y_test, y_pred))

Время выполнения fit: 0.0001 сек.
Время выполнения predict: 2.927 сек.
0.9318181818181818


In [9]:
y_pred_sklearn = KNeighborsClassifier().fit(X_train, y_train).predict(X_test)
print(precision_score(y_test, y_pred_sklearn))

0.9318181818181818
